In [ ]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


c:\Users\supri\anaconda3\anaconda1\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
PINECONE_API_KEY="pcsk_4PPRhS_QHer4UMaB1hjkfL6LfBWVZZSPuaJWKf3xWRfNeQ9PWeJLU9RJvLXsifWjB25n5Z"
PINECONE_ENV="aped-4627-b74a"

In [3]:
def load_pdf(data):
    loader=DirectoryLoader(data,glob='*.pdf',loader_cls=PyPDFLoader)
    docs=loader.load()
    return docs